In [221]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [222]:
#read data
donations = pd.read_csv("contributions.csv")

In [223]:
#drop na rows
donations = donations[pd.notnull(donations['Transaction amt'])]
donations = donations[pd.notnull(donations['Won Primary'])]

#split data based on party

dem = donations[donations['Obama Alum?'].notnull()]
rep = donations[donations['Obama Alum?'].isnull()]

In [224]:


# clean data: replace string values with binary var
dem = dem[['Transaction amt','Won Primary', 'Obama Alum?','Party Support?', 'Emily Endorsed?',
       'Guns Sense Candidate?', 'Biden Endorsed?', 'Warren Endorsed? ',
       'Sanders Endorsed?', 'Our Revolution Endorsed?',
       'Justice Dems Endorsed?', 'PCCC Endorsed?', 'Indivisible Endorsed?',
       'WFP Endorsed?', 'VoteVets Endorsed?', 'No Labels Support?',
        'Rep Party Support?', 'Trump Endorsed?', 'Bannon Endorsed?',
       'Great America Endorsed?', 'NRA Endorsed?', 'Right to Life Endorsed?',
       'Susan B. Anthony Endorsed?', 'Club for Growth Endorsed?',
       'Koch Support?', 'House Freedom Support?', 'Tea Party Endorsed?',
       'Main Street Endorsed?', 'Chamber Endorsed?']].replace(['Yes', 'No'], [1, 0]).fillna(0)




# clean data: replace string values with binary var
rep = rep[['Transaction amt','Won Primary', 'Obama Alum?','Party Support?', 'Emily Endorsed?',
       'Guns Sense Candidate?', 'Biden Endorsed?', 'Warren Endorsed? ',
       'Sanders Endorsed?', 'Our Revolution Endorsed?',
       'Justice Dems Endorsed?', 'PCCC Endorsed?', 'Indivisible Endorsed?',
       'WFP Endorsed?', 'VoteVets Endorsed?', 'No Labels Support?',
        'Rep Party Support?', 'Trump Endorsed?', 'Bannon Endorsed?',
       'Great America Endorsed?', 'NRA Endorsed?', 'Right to Life Endorsed?',
       'Susan B. Anthony Endorsed?', 'Club for Growth Endorsed?',
       'Koch Support?', 'House Freedom Support?', 'Tea Party Endorsed?',
       'Main Street Endorsed?', 'Chamber Endorsed?']].replace(['Yes', 'No'], [1, 0]).fillna(0)


## GLM: Logistic Regression

### A. Democratic Election

#### Features: Transaction amt, Obama Alum, Biden Endorsed, Sanders Endorsed

In [225]:
#split into x and y

#x: try features: Transaction amt, Obama alum, and endorsements from prominent politicians
y = dem['Won Primary']
x = dem[['Transaction amt','Obama Alum?',"Biden Endorsed?","Sanders Endorsed?"]]



In [226]:
#split to train and test sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [227]:
fit_model = sm.Logit(y_train, X_train).fit_regularized()
print(fit_model.summary())

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.5526447289567857
            Iterations: 24
            Function evaluations: 32
            Gradient evaluations: 24
                           Logit Regression Results                           
Dep. Variable:            Won Primary   No. Observations:                 2916
Model:                          Logit   Df Residuals:                     2912
Method:                           MLE   Df Model:                            3
Date:                Mon, 10 May 2021   Pseudo R-squ.:               -0.001291
Time:                        20:12:00   Log-Likelihood:                -1611.5
converged:                       True   LL-Null:                       -1609.4
Covariance Type:            nonrobust   LLR p-value:                     1.000
                        coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------

/usr/local/Cellar/jupyterlab/3.0.14/libexec/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:1810: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/Cellar/jupyterlab/3.0.14/libexec/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:1863: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


In [228]:
predicted = fit_model.predict(X_test)
p = list(map(round, predicted))
print('Test accuracy = ', accuracy_score(y_test, p))

Test accuracy =  0.7534246575342466


## Nonparametric method: Random Forest

In [229]:
forest = RandomForestClassifier(n_estimators=50, max_depth=8)
forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=8, n_estimators=50)

In [230]:
y_pred_test = forest.predict(X_test)
accuracy_score(y_test, y_pred_test)

0.7534246575342466

In [231]:
y_pred_train = forest.predict(X_train)
accuracy_score(y_train, y_pred_train)

0.8024691358024691

In [220]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       199
           1       1.00      1.00      1.00       531

    accuracy                           1.00       730
   macro avg       1.00      1.00      1.00       730
weighted avg       1.00      1.00      1.00       730



#### Features: Obama alum, transaction amt, all endorsement categories

In [214]:
#split into x and y

#x: try features: Transaction amt, Obama alum, and endorsements from prominent politicians

#add some noise to prevent perfect correlation between variables
dem_d = dem+0.00001*np.random.rand(3646, 29)

y = dem['Won Primary']
x = dem_d[['Transaction amt','Won Primary', 'Obama Alum?','Biden Endorsed?','Warren Endorsed? ','Emily Endorsed?']]


#split to train and test sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [215]:

#split to train and test sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)